# 誤差逆伝播法

- ch04で行った、重みパラメータの勾配は数値微分によって求めた。
    - but...数値微分法では計算に時間がかかる
    - --> 効率的に行うために **「誤差伝播法」**
- 誤差伝播法を理解するために...
    - 数式
    - **計算グラフ**

# 計算グラフ

- **計算グラフ**とは？
    - 計算の過程をグラフよってあらわしたもの
    - 複数ノードとエッジによって表現

## 計算グラフで解く
- `p124~`参照
- 計算グラフを使用して問題を解くには...?
    1. 計算グラフを構築
    2. 計算グラフ上で計算を左から右に進める(**順伝播**)
- 順伝播について
    - 計算グラフの出発点から終点への伝播
    - 逆方向は...?
        - **逆伝播**という
        - この動きは、微分を計算する上で重要


## 局所的計算
- 自分に関係している部分だけから、次の結果を出力する
- 全体が複雑であっても、局所的な計算なら単純であり、その結果を伝達することで複雑な計算の結果が得られる

## なぜ計算グラフを使うの？
- 計算グラフの利点：
    - 順伝播と逆伝播によって、各変数の微分の値を効率的に求めることができる
    - Ex)リンゴが1円値上がりしたら、最終的な支出金額は何円増加するか

# 連鎖律

- 逆伝播
    - 右から左に伝達していく
    - 局所的な微分を伝達する原理は**連鎖律**によるもの

## 計算グラフの逆伝播
- `p129~`参照
- Ex)
    - $y = f(x)$ とする
    - $$x → f → y$$
    - $$E\frac{∂y}{∂x} ← f ← E$$
    - 例えば、$y = f(x) = x^2$ のとき、
        - $$\frac{∂y}{∂x} = 2x$$
        - この局所的な微分を上流から伝達された値（E）に乗算して、前のノードに渡す
- ポイント：
    - 目的とする微分の値が簡単に求めることができる
    - これは、連鎖律の原理から説明可能

## 連鎖律とは？
- まずは合成関数から
    - 合成関数＝複数の関数によって構成される関数
    - Ex)  $z = (x + y)^2$ とき、zは２つの式で構成されているといえる
    $$z = t^2$$
    $$t = x + y$$

- **連鎖律の原理**
    - ある関数が合成関数であらわされる場合、その合成関数の微分は合成関数を構成するそれぞれの関数の微分の積によってあらわすことが可能
    - つまり...?
        - $z = (x + y)^2$ であれば、
        $$\frac{∂z}{∂x}は、\frac{∂z}{∂t}と\frac{∂t}{∂x}の積によってあらわせる$$
        $$\frac{∂z}{∂x} = \frac{∂z}{∂t}\frac{∂t}{∂x}$$
- 連鎖率を使って、$\frac{∂z}{∂x}$ を求めてみる
$$\frac{∂z}{∂t} = 2t$$
$$\frac{∂t}{∂x} = 1$$
- よって
$$\frac{∂z}{∂x} = \frac{∂z}{∂t}\frac{∂t}{∂x} = 2t・1 = 2(x + y)$$


## 連鎖率と計算グラフ
- p131～参照

# 逆伝播

## 加算ノードの逆伝播
- 加算の場合は、上流から伝わった微分に**1を乗算**して下流に流す
    - つまり、入力値をそのまま次のノードへ流すだけ
## 乗算ノードの逆伝播
- 乗算の場合は、上流から伝わった微分に、順伝播の際の入力値を **"ひっくり返した値"を乗算**して下流へ流す
    - ひっくり返した値って？
        - 「×ノード」にxとyを入力した場合：
            - 順伝播で`x`の信号であれば逆伝播では`y`
            - 順伝播で`y`の信号であれば逆伝播では`x`
            - と、ひっくり返した関係になっている
        - Ex) $10 × 5 = 50$ の場合
            - 上流から`1.3`の値が流れてくる
            - 10の信号側：1.3 × 5 = 6.5
            - 5の信号側：1.3 × 10 = 13
            - が、それぞれ下流に流れていく
- 逆伝播の値の見方は？
    - 伝播の値をそれぞれｘ、yとする
    - 2つの対象が同じ量だけ増加したら、１つの対象はxの大きさで最終的な結果に影響を与え、もう一方の対象はyの大きさで影響を与えると解釈可能

# 単純なレイヤの実装

- 計算グラフのノードについて
    - 乗算ノード：乗算レイヤ（MulLayer）
    - 加算ノード：加算レイヤ（AddLayer）
- ニューラルネットワークを構築する「層（レイヤ）」を一つのクラスで実装する

## 乗算レイヤの実装
- レイヤは`forward()`と`backward()`という共通のメソッドを持つように実装
    - `forward()`：順伝播
    - `backward()`：逆伝播

In [66]:
class MulLayer:
    def __init__(self):
        """
        インスタンス変数xとyを初期化
        xとyは順伝播の際に入力値を保持するために使用
        """
        self.x = None
        self.y = None

    def forward(self, x, y):
        """
        x, yを受け取り、xとyの積を計算する
        """
        self.x = x
        self.y = y
        out = x * y
        return out

    def backward(self, dout):
        """
        上流から伝わってきた微分（dout）に対して、順伝播の"ひっくり返した値"を乗算する
        """
        dx = dout * self.y  # xとyをひっくり返す
        dy = dout * self.x
        return dx, dy

In [67]:
apple = 100  # りんごの価格
apple_num = 2  # りんごの個数
tax = 1.1  # 税率

# layer
mul_apple_layer = MulLayer()  # りんごの価格と個数を掛けるレイヤー
mul_tax_layer = MulLayer()  # 税率を掛けるレイヤー

# forward(順伝播)
apple_price = mul_apple_layer.forward(apple, apple_num)  # りんごの合計価格
price = mul_tax_layer.forward(apple_price, tax)  # 税込み価格

print(f"forward：\nprice={price}\n")

# backward(逆伝播)
dprice = 1  # 出力の微分（価格に対する微分）
dapple_price, dtax = mul_tax_layer.backward(dprice)  # 税率に対する微分
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # りんごの価格と個数に対する微分
print(f"backward：\ndapple_price= {dapple_price}, \ndtax= {dtax}, \ndapple= {dapple}, \ndapple_num= {dapple_num}")

forward：
price=220.00000000000003

backward：
dapple_price= 1.1, 
dtax= 200, 
dapple= 2.2, 
dapple_num= 110.00000000000001


## 加算レイヤの実装

In [68]:
class AddLayer:
    def __init__(self):
        """
        初期化（何も行わない）
        """
        pass
    
    def forward(self, x, y):
        """
        x, yを受け取り、xとyの和を計算する
        """
        out = x + y
        return out
    
    def backward(self, dout):
        """
        上流から伝わってきた微分（dout）をそのまま返す
        """
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [69]:
apple = 100 # りんごの価格
apple_num = 2 # りんごの個数

orange = 150 # オレンジの価格
orange_num = 3 # オレンジの個数

tax = 1.1 # 税率

# layer
mul_apple_layer = MulLayer()  # りんごの価格と個数を掛けるレイヤー
mul_orange_layer = MulLayer()  # オレンジの価格と個数を掛けるレイヤー
add_apple_orange_layer = AddLayer()  # りんごとオレンジの合計価格を計算するレイヤー
mul_tax_layer = MulLayer()  # 税率を掛けるレイヤー

# forward(順伝播)
apple_price = mul_apple_layer.forward(apple, apple_num)  # りんごの合計価格
orange_price = mul_orange_layer.forward(orange, orange_num)  # オレンジの合計価格
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # りんごとオレンジの合計価格
price = mul_tax_layer.forward(all_price, tax)  # 税込み価格
print(f"Price：{price}\n")

# backward(逆伝播)
dprice = 1  # 出力の微分（価格に対する微分）
dall_price, dtax = mul_tax_layer.backward(dprice)  # 税率に対する微分
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # りんごとオレンジの合計価格に対する微分
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # りんごの価格と個数に対する微分
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # オレンジの価格と個数に対する微分
print(f"Backward:\ndall_price= {dall_price}, \ndtax= {dtax}, \ndapple_price= {dapple_price}, \ndorange_price= {dorange_price}, \ndapple= {dapple}, \ndapple_num= {dapple_num}, \ndorange= {dorange}, \ndorange_num= {dorange_num}")

Price：715.0000000000001

Backward:
dall_price= 1.1, 
dtax= 650, 
dapple_price= 1.1, 
dorange_price= 1.1, 
dapple= 2.2, 
dapple_num= 110.00000000000001, 
dorange= 3.3000000000000003, 
dorange_num= 165.0


# 活性化関数レイヤの実装

- 計算グラフの考え方をニューラルネットワークに適用する

## ReLUレイヤ
- 活性化関数として使用されるReLU
    - 入力xが0より大きければ、そのままその値を返す
    - 入力yが0以下であれば、0を返す
```math
f(x) = \left\{
\begin{array}{ll}
x & (x > 0) \\
0 & (x ≦ 0)
\end{array}
\right.
```
- xに関するyの微分の式は以下のようになる
```math
\frac{∂y}{∂x} = \left\{
\begin{array}{ll}
1 & (x > 0) \\
0 & (x ≦ 0)
\end{array}
\right.
```
- 上記の式から...
    - 順伝播時の入力である**xが0より大きい**時：
        - 逆伝播は上流の値をそのまま下流に流す
    - 順伝播時の入力である**xが0以下**の時：
        - 逆伝播では、下流への信号はストップする（0が流れて終わり）

- 実装
    - 入力はNumPyの配列が入力される


In [70]:
import numpy as np
class Relu:
    def __init__(self):
        """
        True/Falseのマスクを保持するためのインスタンス変数maskを初期化
        """
        self.mask = None

    def forward(self, x):
        """
        xを受け取り、ReLU関数を適用する
        x：入力値（NumPy配列）
        出力：ReLU関数を適用した結果（NumPy配列）
        """
        self.mask = (x <= 0)  # xが0以下の要素をTrueにするマスクを作成
        out = x.copy()
        out[self.mask] = 0  # マスクされた要素を0にする
        return out
    
    def backward(self, dout):
        """
        上流から伝わってきた微分（dout）に対して、ReLUのマスクを適用する
        dout：上流からの微分（NumPy配列）
        出力：ReLU関数の微分結果（NumPy配列）
        """
        dout[self.mask] = 0  # マスクされた要素に対しては微分を0にする
        dx = dout
        return dx

## Sigmoidレイヤ
- シグモイド関数の関数
$$y=\frac{1}{1+exp(-x)}$$
- 計算グラフでは、順伝播の場合は上流から順に以下のようになる
    ```math
    \begin{aligned}
    &\text{「×ノード」} \quad x \times -1 = -x \\
    &\text{「expノード」} \quad \exp(-x) \\
    &\text{「+ノード」} \quad \exp(-x) + 1 \\
    &\text{「/ノード」} \quad \frac{1}{1+\exp(-x)} = y
    \end{aligned}
    ```

- 逆伝播の流れ
    1. 「/ノード」$y = \frac{1}{x}$ の微分
        $$\frac{∂y}{∂x} = -\frac{1}{x^2} = -y^2$$
        - 上流の値に対して $-y^2$ を乗算して下流へ伝播
    2. 「+ノード」
        - 上流の値をそのまま下流へ流す
    3. 「expノード」$y = exp(x)$ の微分
        $$\frac{∂y}{∂x} = exp(x)$$
        - 上流の値に対して`exp(x)`を乗算して下流へ伝播
    4. 「×ノード」
        - "ひっくり返した値"を乗算
        - x側を求めるには、上流の値に対して`-1`を乗算して下流へ伝播
- 上記の逆伝播の流れより、上流の微分した値が </br> 
    $$\frac{∂L}{∂y}$$  
    であった場合、ステップ1~4を経ると、
    1. $$-\frac{∂L}{∂y}y^2$$
    2. $$-\frac{∂L}{∂y}y^2$$
    3. $$-\frac{∂L}{∂y}y^2exp(x)$$
    4. $$\frac{∂L}{∂y}y^2exp(x)$$
    となる。
- ここで、上記の式を整理すると...
```math
\frac{∂L}{∂y}y^2exp(x) = \frac{∂L}{∂y}\frac{1}{(1+exp(-x))^2}exp(x) \\
\hspace{75pt}= \frac{∂L}{∂y}\frac{1}{1+exp(-x)}\frac{exp(x)}{1+exp(x)} \\
= \frac{∂L}{∂y}y(1-y)
```
- つまり、Sigmoidレイヤに上流の値を通すと中身はあまり考えなくとも、上記の式の出力だけに集中することができる。

In [71]:
import numpy as np
class Sigmoid:
    def __init__(self):
        """
        シグモイド関数の出力を保持するためのインスタンス変数outを初期化
        """
        self.out = None
    
    def forward(self, x):
        """
        シグモイド関数を適用する
        x：入力値（NumPy配列）
        出力：シグモイド関数を適用した結果（NumPy配列）
        """
        out = 1 / (1 + np.exp(-x))  # シグモイド関数の計算
        self.out = out
        return out
    
    def backward(self, dout):
        """
        上流から伝わってきた微分（dout）に対して、シグモイド関数の微分を適用する
        """
        dx = dout * self.out * (1 - self.out)  # シグモイド関数の微分
        return dx

# Affine/Softmaxレイヤの実装

## Affineレイヤ
- ニューラルネットワークの順伝播
    - 行列の内積を用いた
    - ニューロンの重み付き和は、`Y = np.dot(X, W) + B`と計算できる
        - この時、対応する次元数は一致させる必要がある
        - (2,)<->(2,3)<->(3,)
    - そしてこの`Y`が活性化関数によって変換され、次の層へと伝播する
    - このようなニューラルネットワークの順伝播で行う行列の内積は、「**アフィン変換**」と呼ばれる
    

In [72]:
import numpy as np
X = np.random.rand(2)  # 要素数2つ（2列）入力データ
W = np.random.rand(2, 3)  # 2x3の重み行列
b = np.random.rand(3)  # 3次元のバイアス
print(X)
print(W)
print(b)
print("Xの形状:", X.shape)
print("Wの形状:", W.shape)
print("bの形状:", b.shape)

# 内積を求める
Y = np.dot(X, W) + b  # XとWの内積にバイアスbを加える
print("Y:", Y)

[0.16487319 0.58380276]
[[0.84589124 0.50932981 0.28588086]
 [0.89121406 0.75717278 0.61157984]]
[0.64316609 0.35932206 0.37141972]
Xの形状: (2,)
Wの形状: (2, 3)
bの形状: (3,)
Y: [1.30292411 0.88533645 0.77559581]


- ここで、順伝播である行列の内積とバイアスの和を計算グラフであらわす
    - 内積は「dotノード」として表現
    - ()内は、それぞれの次元をあらわす
    - X, W, Bはそれぞれ入力値、重み、バイアスをあらわし、すべて**行列**である
    ```math
    \begin{aligned}
    &\text{「dotノード」} \quad dot(X(2,), W(2, 3)) = X・W(3,)\\
    &\text{「+ノード」} \quad X・W(3,) + B(3,) = Y(3,) \\
    \end{aligned}
    ```

- 次に、逆伝播について考える(p149~参照)
    - 行列の逆伝播を求める時、要素ごとに考えると良い
    - 次の式が得られる
        ```math
        \frac{∂L}{∂X} = \frac{∂L}{∂Y}・W^T\\
        \frac{∂L}{∂W} = X^T・\frac{∂L}{∂Y}
        ```
    - Tは転置を表現
        - 転置：Wの(i, j)の要素を(j, i)の要素に入れ替えること
    1. 「+ノード」
        - そのまま流す
    2. 「dotノード」
        ```math
        \frac{∂L}{∂X} = \frac{∂L}{∂Y}・W^T\\
        \frac{∂L}{∂W} = X^T・\frac{∂L}{∂Y}
        ```
    
- この辺ムズイ(ーー;)p149~の図をしっかり参照すること

## バッチ版Affineレイヤ
- N個のデータをまとめて順伝播する
    - データのまとまり＝「バッチ」
- １つのデータの時との違い：
    - 入力データが(N, 2)に変更
    - それ以外は、すべて同じ！
- 注意点：**バイアスの加算**
    - バイアスの加算は、それぞれのデータに加算される
    - これにより、逆伝播の際は、それぞれのデータの逆伝播の値がバイアスの要素に集約される必要がある

In [73]:
x = np.array([[0.0, 0.0, 0.0],[10, 10, 10]]) 
b = np.array([1.0, 2.0, 3.0])
print(x + b)  # ブロードキャストにより、bがxの各データに加算される

dY = np.array([[1, 2, 3],[4, 5, 6]])  # 出力の微分（dY）を初期化
db = np.sum(dY, axis=0)  # バイアスの微分（db）を計算
print(db)

[[ 1.  2.  3.]
 [11. 12. 13.]]
[5 7 9]


In [74]:
import numpy as np
class Affine:
    def __init__(self, W, b):
        """
        重みW、バイアスb、入力xを初期化
        """
        self.W = W
        self.b = b
        self.x = None
        self.dW = None  # 重みの微分
        self.db = None  # バイアスの微分

    def forward(self, x):
        """
        xを受け取り、重みWとバイアスbを用いて内積を計算する
        x：入力値（NumPy配列）
        出力：線形変換の結果（NumPy配列）
        """
        self.x = x  # 入力値を保持
        out = np.dot(x, self.W) + self.b  # 内積とバイアスの加算
        return out
    
    def backward(self, dout):
        """
        上流から伝わってきた微分（dout）に対して、重みWとバイアスbの微分を計算する
        dout：上流からの微分（NumPy配列）
        出力：入力xの微分（NumPy配列）
        """
        dx = np.dot(dout, self.W.T)  # 入力の微分
        self.dW = np.dot(self.x.T, dout)  # 重みの微分
        self.db = np.sum(dout, axis=0)  # バイアスの微分
        return dx

## Softmax-with-Lossレイヤ
- 最後に出力層であるソフトマックス関数について
    - 入力された値を正規化して出力
    - 合計が１になるように変形して出力
- 「Softmax関数」＋「損失関数である交差エントロピー誤差」＝「Softmax-with-Loss」
    - ソフトマックス関数（Softmaxレイヤ）は、入力値(a_n)を正規化した値(y_n)を出力
    - 交差エントロピー誤差（Cross Entropy Errorレイヤ）は、ソフトマックスの出力(y_n)と、教師ラベル(t_n)を受け取りそれらのデータから損失Lを出力する
- 逆伝播について
    - Softmaxレイヤから逆伝播は、`(y_1 - t_1, y_2 - t_2, y_3 -t_3)`という結果になる
        - かなり"きれい"な状態になる！！
            - これは、交差エントロピー誤差という関数の性質
        - これは、Softmaxレイヤの出力と教師ラベルの差分である
        - この差分である誤差が前レイヤへ伝わっていく
            - **重要な性質！！**
- ニューラルネットワークの学習の目的：
    - 出力を今日調べるに近づけるように、重みパラメータを調整すること
    - ニューラルネットワークの出力と教師ラベルとの誤差を効率的に前レイヤに伝える必要がある
    - `(y_1 - t_1, y_2 - t_2, y_3 -t_3)`という結果は、ソフトマックス関数の出力と教師ラベルの差！！
        - ニューラルネットワークの出力と教師ラベルの差を表現
    - Ex)
        - 認識誤差が大きい場合
            - 教師ラベル：（0, 1, 0）
            - Softmaxレイヤの出力：(0.3, 0.2, 0.5)
            - 正解ラベルに対する確率は20%であり、正しい認識ができていない
            - この時の逆伝播は、(0.3-0, 0.2-1.0, 0.5-0)=(0.3, -0.8, 0.5)と大きな誤差を伝播可能
        - 認識誤差が小さい場合
            - 教師ラベル：(0, 1, 0)
            - Softmaxxレイヤの出力：(0.01, 0.99, 0)
            - 正解ラベルに対する確率は99%であり、ほぼ正しい認識ができている
            - この時の逆伝播は、(0.01-0, 0.99-1, 0-0)=(0.01, -0.01. 0)と非常に小さな誤差になる


In [75]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリをパスに追加
from deep_learning_from_scratch.common.functions import *

import numpy as np

class SoftmaxWithLoss:
    def __init__(self):
        """
        ソフトマックス関数の出力と正解ラベルを保持するためのインスタンス変数
        """
        self.loss = None  # 損失値
        self.y = None  # ソフトマックス関数の出力
        self.t = None  # 正解ラベル（One-hotエンコーディングされた形式）

    def forward(self, x, t):
        """
        ソフトマックス関数を計算し、クロスエントロピー損失を求める
        x：入力値（NumPy配列）
        t：正解ラベル（NumPy配列）
        出力：損失値
        """
        self.t = t  # 正解ラベルを保持
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        """
        損失の微分を計算する
        dout：上流からの微分（デフォルトは1）
        出力：ソフトマックス関数の出力に対する微分
        """
        batch_size = self.t.shape[0]  # バッチサイズを取得
        dx = (self.y - self.t) / batch_size  # データ１個あたりの誤差が前レイヤーに伝播する
        return dx
    

# 誤差逆伝播法の実装

## ニューラルネットワークの学習の全体図
- 前提  
    - 「学習」：
        - 重みとバイアスのパラメータを訓練データに適応するように調整すること
- ステップ1（ミニバッチ）
    - 訓練データの中から無作為に一部のデータを選び出す
- ステップ2（勾配の算出）
    - 各重みパラメータに関する損失関数の勾配を求める
- ステップ3（パラメータの更新）
    - 重みパラメータを勾配方向に微小量だけ更新する
- ステップ4（繰り返す）
    - ステップ1~3を繰り返す    

## 誤差逆伝播法に対応したニューラルネットワークの実装
- TwoLayerNet
    - 2層のニューラルネットワーク

In [ ]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリをパスに追加
from deep_learning_from_scratch.common.functions import *
from deep_learning_from_scratch.common.gradient import numerical_gradient  # 勾配を計算する関数

import numpy as np
from collections import OrderedDict  # 順序付き辞書を使用, 追加した要素の順序を覚えることが可能

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        """
        二層のニューラルネットワークを初期化する
        input_size：入力層のサイズ
        hidden_size：隠れ層のサイズ
        output_size：出力層のサイズ
        weight_init_std：重みの初期化標準偏差（デフォルトは0.01）
        """
        # 重みとバイアスを初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤーを初期化
        self.layers = OrderedDict()  # 順序付き辞書を使用してレイヤーを保持
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])  # Affineレイヤーを追加
        self.layers['Relu1'] = Relu()  # ReLUレイヤーを追加
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])  # 2つ目のAffineレイヤーを追加
        
        # 損失関数を初期化
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        """
        入力xに対して予測を行う
        x：入力データ（NumPy配列）
        出力：予測結果（NumPy配列）
        """
        for layer in self.layers.values():
            x = layer.forward(x)

        return x
    
    def loss(self, x, t):
        """
        入力xと正解ラベルtに対して損失を計算する
        x：入力データ（NumPy配列）
        t：正解ラベル（NumPy配列）
        出力：損失値
        """
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        """
        入力xと正解ラベルtに対して精度を計算する
        x：入力データ（NumPy配列）
        t：正解ラベル（NumPy配列）
        出力：精度（0から1の範囲）
        """
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)  # tがOne-hotエンコーディングされている場合、インデックスに変換
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        """
        入力xと正解ラベルtに対して、パラメータの数値微分を計算する
        x：入力データ（NumPy配列）
        t：正解ラベル（NumPy配列）
        出力：パラメータの勾配（辞書形式）
        """
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])  # W1の勾配を計算
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])  # b1の勾配を計算
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])  # W2の勾配を計算
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])  # b2の勾配を計算
        return grads
    
    def gradient(self, x, t):
        """
        入力xと正解ラベルtに対して、パラメータの勾配を計算する
        x：入力データ（NumPy配列）
        t：正解ラベル（NumPy配列）
        出力：パラメータの勾配（辞書形式）
        """
        # 順伝播(forward)
        self.loss(x, t)

        # 逆伝播(backward)
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        return grads

## 誤差逆伝播法の勾配確認
- 勾配を求める方法は２つ
    1. 数値微分によって求める
        - 単純
        - 非効率的
    2. 解析的に数式を解いて求める
        - 複雑
        - 誤差逆伝播法を用いることで、大量のパラメータが存在しても効率的に計算可能
- 数値微分の必要性
    - 数値微分は、非効率的なら誤差逆伝播法だけでいいのでは...?
    - **誤差逆伝播法の正しさを確認するための使用**できる！！
        - 数値微分は、単純な構造なのでミスしにくい
    - この誤差逆伝播法の勾配結果と数値微分の勾配結果がほぼ一致することを確認する作業を**勾配確認**という


In [ ]:
import sys, os
sys.path.append(os.pardir) 
from deep_learning_from_scratch.dataset.mnist import load_mnist  # MNISTデータセットを読み込む関数

import numpy as np

# MNISTデータセットを読み込む
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)  # ネットワークを初期化

x_batch = x_train[:3]  # バッチサイズ3の入力データを取得(3, 784)
t_batch = t_train[:3]  # バッチサイズ3の正解ラベルを取得 (3,)

grad_numerical = network.numerical_gradient(x_batch, t_batch)  # 数値微分を計算
grad_backprop = network.gradient(x_batch, t_batch)  # 逆伝播を使用して勾配を計算


# 各重みの絶対誤差の平均を求める
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_numerical[key] - grad_backprop[key]))
    print(f"{key}の絶対誤差の平均: {diff}")  # 絶対誤差の平均を表示

W1の絶対誤差の平均: 5.730597504455014e-10
b1の絶対誤差の平均: 3.6196919858059572e-09
W2の絶対誤差の平均: 6.917187013379909e-09
b2の絶対誤差の平均: 1.4030637523892996e-07


- 上記の結果から、数値微分と誤差逆伝播法でそれぞれ求めた勾配の差葉かなり小さいことがわかる！

## 誤差逆伝播法を使った学習
- 誤差逆伝播法を使ったニューラルネットワークの学習の実装を行う

In [86]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリをパスに追加
from deep_learning_from_scratch.dataset.mnist import load_mnist  # MNISTデータセットを読み込む関数

import numpy as np

# MNISTデータセットを読み込む
(img_train, label_train), (img_test, label_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)  # ネットワークを初期化

# 学習の設定
iters_num = 10000  # 学習の繰り返し回数
train_size = img_train.shape[0]  # 学習データのサイズ
batch_size = 100  # バッチサイズ
learning_rate = 0.1  # 学習率

# 学習のための変数を初期化
train_loss_list = []  # 損失値を保存するリスト
train_acc_list = []  # 学習データの精度を保存するリスト
test_acc_list = []  # テストデータの精度を保存するリスト

iter_per_epoch = max(train_size / batch_size, 1)  # 1エポックあたりのイテレーション数

for i in range(iters_num):
    # バッチの取得
    batch_mask = np.random.choice(train_size, batch_size)  # ランダムにバッチサイズ分のインデックスを取得
    x_batch = img_train[batch_mask]  # バッチの入力データ
    t_batch = label_train[batch_mask]  # バッチの正解ラベル

    # 勾配を計算
    grads = network.gradient(x_batch, t_batch)

    # パラメータの更新
    for key in network.params.keys():
        network.params[key] -= learning_rate * grads[key]

    # 損失値を計算
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 精度を計算
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(img_train, label_train)  # 学習データの精度
        test_acc = network.accuracy(img_test, label_test)  # テストデータの精度
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f"イテレーション {i}, 損失: {loss}, 学習精度: {train_acc}, テスト精度: {test_acc}")


イテレーション 0, 損失: 2.301524659657293, 学習精度: 0.09683333333333333, テスト精度: 0.0979
イテレーション 600, 損失: 0.39448905606157036, 学習精度: 0.9010166666666667, テスト精度: 0.9038
イテレーション 1200, 損失: 0.24767847017670683, 学習精度: 0.9231333333333334, テスト精度: 0.9267
イテレーション 1800, 損失: 0.17545380701935656, 学習精度: 0.9367333333333333, テスト精度: 0.9352
イテレーション 2400, 損失: 0.12473206122853889, 学習精度: 0.9448333333333333, テスト精度: 0.9434
イテレーション 3000, 損失: 0.07413857703530645, 学習精度: 0.9534666666666667, テスト精度: 0.9513
イテレーション 3600, 損失: 0.12077940488068836, 学習精度: 0.9575666666666667, テスト精度: 0.9548
イテレーション 4200, 損失: 0.06904663794774413, 学習精度: 0.9626166666666667, テスト精度: 0.9606
イテレーション 4800, 損失: 0.0613175648152332, 学習精度: 0.9648833333333333, テスト精度: 0.9612
イテレーション 5400, 損失: 0.0923907582159141, 学習精度: 0.96815, テスト精度: 0.9628
イテレーション 6000, 損失: 0.10574318820933709, 学習精度: 0.97125, テスト精度: 0.9655
イテレーション 6600, 損失: 0.10109669456913004, 学習精度: 0.9734166666666667, テスト精度: 0.9669
イテレーション 7200, 損失: 0.11210692564866402, 学習精度: 0.976, テスト精度: 0.9685
イテレーション 7800, 損